# TensorFlow BYOM: Treinando local e efetuando o deploy no Amazon SageMaker


## Introdução 

Vamos primeiramente treinar um modelo localmente, sem enviar o treinamento para o SageMaker. Após a criação do modelo, vamos efetuar o deploy no endpoint de inferência do Amazon SageMaker.

Iremos utilizar o ``tensorflow.estimator.DNNClassifier`` no dataset [IRIS](https://archive.ics.uci.edu/ml/datasets/iris).

In [ ]:
import os 
import boto3, re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow.model import TensorFlowModel
from iris_dnn_classifier import estimator_fn
from six.moves.urllib.request import urlopen
from iris_dnn_classifier import train_input_fn
from iris_dnn_classifier import serving_input_fn

role = get_execution_role()

### Definição do Modelo

Vamos utilizar o [``tensorflow.estimator.DNNClassifier``](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier) como Estimator. Também precisamos escrever alguns métodos para fornecer entradas durante a hospedagem e o treinamento. Esses métodos são todos encontrados abaixo.

In [ ]:
!cat iris_dnn_classifier.py

In [ ]:
classifier = estimator_fn(run_config = None, params = None)

### Setup dos dados

Em seguida, precisamos extrair os dados do repositório Tensorflow e torná-los prontos para o treinamento.

In [ ]:
# Datasets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
      f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
      f.write(raw)

train_func = train_input_fn('.', params = None)

### Treinamento

In [ ]:
classifier.train(input_fn = train_func, steps = 1000)

## Preparando o modelo para hospedagem

### Exportando o modelo do Tensorflow

Para configurar o hosting, temos que importar o modelo do treinamento para o hosting. Começaremos exportando o modelo do TensorFlow e salvando-o, mas uma estrutura precisa ser seguida. O modelo exportado deve ser convertido em um formato legível por `` sagemaker.tensorflow.model.TensorFlowModel``. O código a seguir descreve a exportação do modelo:

Há uma pequena diferença entre um modelo SageMaker e um modelo TensorFlow. A conversão é fácil e bastante trivial. Simplesmente mova o modelo exportado do tensorflow para um diretório `` export/Servo/`` e comprima em tar o diretório inteiro. O SageMaker reconhecerá isso como um modelo do TensorFlow.

In [ ]:
exported_model = classifier.export_savedmodel(export_dir_base = 'export/Servo/', 
                               serving_input_receiver_fn = serving_input_fn)

print (exported_model)
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

### Importando o modelo no Amazon SageMaker

Iremos utilizar o mesmo bucket da etapa anterior para armazenar nosso modelo.

In [ ]:
bucket= 'sagemaker-workshop-files-us-east-1-206730628141' # troque pelo seu bucket (o bucket foi criado pelo CloudFormation, verifique a aba de output no CloudFormation)
def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, 'rb')
    key = '{}/{}'.format(channel, file)
    s3.Bucket(bucket).put_object(Key=key, Body=data)

In [ ]:
upload_to_s3('tf-model','model.tar.gz')

In [ ]:
sagemaker_model = TensorFlowModel(model_data = 's3://' + bucket + '/tf-model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'iris_dnn_classifier.py')

### Criando o endpoint no Amazon SageMaker

Podemo usar o ``sagemaker.tensorflow.model.TensorFlowModel.deploy`` como método para efetuar o deploy do modelo.

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                          instance_type='ml.m5.xlarge')

### Efetuando chamadas para o endpoint

In [ ]:
sample = [6.4,3.2,4.5,1.5]
predictor.predict(sample)

Exclua todos os diretórios temporários para que não afetemos a próxima execução.

In [ ]:
os.remove('model.tar.gz')
import shutil
shutil.rmtree('export')

Se você não quiser continuar usando o endpoint, pode removê-lo.

In [ ]:
# sagemaker.Session().delete_endpoint(predictor.endpoint)